# Création du Dataframe + initialisation de la Data

Pensez a gerer les colonnes en trop excel (df.drop) \
Pour chimiste 2 mso trop long penser a tout mettre dans 1 case \
A faire je pense gestion du profile du gars du type bouton `Mon Profil` avec possibilité de changer année groupe filiere et Mso

In [1]:
import pandas as pd
import json
from numpyencoder import NumpyEncoder

# Read csv file
Df = pd.read_csv('./planning_Csv/planningTestV3.csv', sep=';')
# Filter columns and rows
#Df = Df[['LUNDI', 'MARDI', 'MERCREDI', 'JEUDI', 'VENDREDI']]
Df.reset_index()
Df.fillna(0, inplace=True)
Df

,LUNDI,MARDI,MERCREDI,JEUDI,VENDREDI,Unnamed: 5
0,03-avr,04-avr,05-avr,06-avr,07-avr,0.0
1,8h-12h15,8h-12h15,8h-12h15,8h-12h15,8h-12h15,0.0
2,CONCEP.LOGICIELLE/BIG DATA,GROUPE A,GROUPE A & B,GROUPE C,CONCEP.LOGICIELLE/BIG DATA,0.0
3,0,0,0,0,0,0.0
4,0,Projet,Projet,Projet,0,0.0
...,...,...,...,...,...,...
65,0,0,0,0,Salle info,0.0
66,0,0,0,0,0,0.0
67,0,0,0,0,0,0.0
68,0,0,0,0,0,0.0


### On récupère la liste de toutes les matière/ prof/ majeurs dans LstSemaine

In [2]:
def createSchedule(Df):
    DicoJour = {}
    IsAprem = False
    LstSemaine = []
    # parcour des jours pour séparer matin et aprem
    for obj in Df :
        LstMatin = []
        LstAprem = []
        LstTemp = []
        # parcour des cases non nulles 
        for case in Df[obj] :
            if case != 0 :
                LstTemp.append(case)

        # parcour des cases non nulles d'une journée pour la séparation
        for index,value in enumerate(LstTemp) :
            if index == 0 :
                # on ajoute le jour
                DicoJour["jour"] = obj +"-" + value
            else :
                if value in {'13h-14h30','13h30-17h45','13h10-18h', '13h30-15h30', '15h45-17h45', '13h30 - 14h30','13h30 - 17h45','13h30 - 17h30'} :
                  IsAprem = True
                elif value in {'8h-12h15','8h - 12h15', '8h30 - 10h00', '8h-10h', '10h15-12h15','10h15 - 12h15', '8h- 10h'}:
                    IsAprem = False
                
                if IsAprem :
                    LstAprem.append(value)
                else :
                    LstMatin.append(value)

        DicoJour["Matin"] = LstMatin.copy()
        DicoJour["Aprem"] = LstAprem.copy()
        LstSemaine.append(DicoJour.copy())
    return LstSemaine

LstSemaine = createSchedule(Df)
print(LstSemaine)
df_semaine = pd.DataFrame( LstSemaine)
df_semaine
# df_semaine.to_json('planning_semaineMatin_Aprem.json',orient='records', indent=4)

[{'jour': 'LUNDI-03-avr', 'Matin': ['8h-12h15', 'CONCEP.LOGICIELLE/BIG DATA', 'Pas de cours ', 'ROBOTIQUE', 'Pas de cours ', 'ELECTRONIQUE ET SYST EMB', 'Communications numériques', 'N.LEBEDEV', ' Salle ', '08h00 à 10h00', 'INFRA DES RESEAUX ', 'Communications numériques', 'N.LEBEDEV', ' Salle ', '08h00 à 10h00', 'IMAGE ', "Traitement d'images ", 'E. VAN REETH /M.FOARE ', '10h30 à 12h30 :  Salleinfo '], 'Aprem': ['13h10-18h', 'LV', '(Voir planning)']}, {'jour': 'MARDI-04-avr', 'Matin': ['8h-12h15', 'GROUPE A', 'Projet ', 'F. JOLY', 'Salle info', 'CONCEP.LOGICIELLE/BIG DATA', 'GROUPE C &D', 'Conception Orientée Objet et Design Patterns', 'B.MASCRET', ' Salle info'], 'Aprem': ['13h30-17h45', 'GROUPE C&D', 'Projet ', 'F. JOLY', 'Salles infos', 'CONCEP.LOGICIELLE/BIG DATA', 'GROUPE A&B', 'Conception Orientée Objet et Design Patterns', 'B.MASCRET', ' Salle info']}, {'jour': 'MERCREDI-05-avr', 'Matin': ['8h-12h15', 'GROUPE A & B', 'Projet ', 'F. JOLY', 'Salles info', 'ROBOTIQUE', 'GROUPE C&D

,jour,Matin,Aprem
0,LUNDI-03-avr,"[8h-12h15, CONCEP.LOGICIELLE/BIG DATA, Pas de ...","[13h10-18h, LV, (Voir planning)]"
1,MARDI-04-avr,"[8h-12h15, GROUPE A, Projet , F. JOLY, Salle i...","[13h30-17h45, GROUPE C&D, Projet , F. JOLY, Sa..."
2,MERCREDI-05-avr,"[8h-12h15, GROUPE A & B, Projet , F. JOLY, Sal...","[13h30-17h45, CONCEP.LOGICIELLE/BIG DATA, Pas ..."
3,JEUDI-06-avr,"[8h-12h15, GROUPE C, Projet , F. JOLY, Salle i...","[13h30-17h45, TP TSA , Séance de Rattrapages ,..."
4,VENDREDI-07-avr,"[8h-12h15, CONCEP.LOGICIELLE/BIG DATA, Pas de ...","[13h30-17h45, MSO1 OUVERTURE, AWS Cloud Founda..."
5,VENDREDI-07-avr,[],[]


### On filtre les Majeurs par jour/ Demi-journée et majeurs ou tronc commun 4ETI

In [3]:
def genDic(lstkey) :
    Dico = {}
    for key in lstkey :
        Dico[key] = []
    return Dico

test = genDic(["Pour tous","CONCEP.LOGICIELLES/BIG DATA"])
print (test)

{'Pour tous': [], 'CONCEP.LOGICIELLES/BIG DATA': []}


In [4]:
def filtreMaj(LstSemaine) :
    Semaine =  {}
    for jour in LstSemaine :
        Majeur = "Pour tous" #on initialise le majeur à tous
        Mat= {"CONCEP.LOGICIELLE/BIG DATA":[],
        "ROBOTIQUE":[],
        "ELECTRONIQUE ET SYST EMB":[],
        "INFRA DES RESEAUX":[],
        "IMAGE":[],
        "Pour tous":[]} #on initialise les majeurs
        Semaine[jour["jour"]] = {}
        for Case in jour["Matin"] : #on parcours les cours du matin et on les ajoute au majeur correspondant 
            if Case == "CONCEP.LOGICIELLE/BIG DATA" :
                Majeur = "CONCEP.LOGICIELLE/BIG DATA"
            elif Case == "ROBOTIQUE" :
                Majeur = "ROBOTIQUE"
            elif Case == "ELECTRONIQUE ET SYST EMB" :
                Majeur = "ELECTRONIQUE ET SYST EMB"
            elif Case == "INFRA DES RESEAUX " :
                Majeur = "INFRA DES RESEAUX"
            elif Case == "IMAGE " :
                Majeur = "IMAGE"
            Mat[Majeur].append(Case)
            
        Semaine[jour["jour"]]["Matin"] = Mat
        Majeur = "Pour tous" #on remet le majeur par défaut
        Aprem= {"CONCEP.LOGICIELLE/BIG DATA":[],
        "ROBOTIQUE":[],
        "ELECTRONIQUE ET SYST EMB":[],
        "INFRA DES RESEAUX":[],
        "IMAGE":[],
        "Pour tous":[]} #on remet le majeur par défaut
        for Case in jour["Aprem"] : #on parcours les cours de l'aprem et on les ajoute au majeur correspondant
            if Case == "CONCEP.LOGICIELLE/BIG DATA" :
                Majeur = "CONCEP.LOGICIELLE/BIG DATA"
            elif Case == "ROBOTIQUE" :
                Majeur = "ROBOTIQUE"
            elif Case == "ELECTRONIQUE ET SYST EMB" :
                Majeur = "ELECTRONIQUE ET SYST EMB"
            elif Case == "INFRA DES RESEAUX " :
                Majeur = "INFRA DES RESEAUX"
            elif Case == "IMAGE " :
                Majeur = "IMAGE"
            
            Aprem[Majeur].append(Case)
        Semaine[jour["jour"]]["Aprem"] = Aprem
    return Semaine
Planning = filtreMaj(LstSemaine)
print(Planning)

{'LUNDI-03-avr': {'Matin': {'CONCEP.LOGICIELLE/BIG DATA': ['CONCEP.LOGICIELLE/BIG DATA', 'Pas de cours '], 'ROBOTIQUE': ['ROBOTIQUE', 'Pas de cours '], 'ELECTRONIQUE ET SYST EMB': ['ELECTRONIQUE ET SYST EMB', 'Communications numériques', 'N.LEBEDEV', ' Salle ', '08h00 à 10h00'], 'INFRA DES RESEAUX': ['INFRA DES RESEAUX ', 'Communications numériques', 'N.LEBEDEV', ' Salle ', '08h00 à 10h00'], 'IMAGE': ['IMAGE ', "Traitement d'images ", 'E. VAN REETH /M.FOARE ', '10h30 à 12h30 :  Salleinfo '], 'Pour tous': ['8h-12h15']}, 'Aprem': {'CONCEP.LOGICIELLE/BIG DATA': [], 'ROBOTIQUE': [], 'ELECTRONIQUE ET SYST EMB': [], 'INFRA DES RESEAUX': [], 'IMAGE': [], 'Pour tous': ['13h10-18h', 'LV', '(Voir planning)']}}, 'MARDI-04-avr': {'Matin': {'CONCEP.LOGICIELLE/BIG DATA': ['CONCEP.LOGICIELLE/BIG DATA', 'GROUPE C &D', 'Conception Orientée Objet et Design Patterns', 'B.MASCRET', ' Salle info'], 'ROBOTIQUE': [], 'ELECTRONIQUE ET SYST EMB': [], 'INFRA DES RESEAUX': [], 'IMAGE': [], 'Pour tous': ['8h-12h1

### Gestion des 4CGP

In [7]:
def filtre4CGP(LstSemaine) :
    Semaine =  {}
    for jour in LstSemaine :
        print(jour["jour"])
        Majeur = "Pour tous" #on initialise le majeur à tous
        Mat= {"Groupe A":[],
        "Groupe B":[],
        "Groupe C":[],
        "Chimie Médicinale et Hétérocycles":[],
        "Génie de la réaction chimique avancée":[],
        "Transition Energétique":[],
        "Analyse en ligne" : [],
        "Synthèse Macromoléculaire":[],
        
        "Pour tous":[]} #on initialise les majeurs

filtre4CGP(LstSemaine)

LUNDI-22-mars
MARDI -23-mars
MERCREDI-24-mars
JEUDI-25-mars


In [5]:
with open('./Output_Json/Datatest.json', 'w+') as f:
    json.dump(Planning, f, indent=4, cls=NumpyEncoder)

